In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset
df = pd.read_csv('Churn_Modelling.csv')  # Adjust the path as necessary

# Drop columns that are not relevant for modeling
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

# Apply one-hot encoding to categorical features
df = pd.get_dummies(df, columns=['Geography', 'Gender'], drop_first=True)

# Assuming 'Exited' is the correct target variable column
X = df.drop('Exited', axis=1)
y = df['Exited'].values

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomerLeavePredictor(nn.Module):
    def __init__(self):
        super(CustomerLeavePredictor, self).__init__()
        self.fc1 = nn.Linear(11, 20)  # 11 features, 20 neurons in first hidden layer
        self.fc2 = nn.Linear(20, 20) # 20 neurons in second hidden layer
        self.output = nn.Linear(20, 1) # Output layer

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.output(x))
        return x

In [ ]:
model = CustomerLeavePredictor()
criterion = nn.BCELoss() # Binary Cross Entropy Loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 1000
for epoch in range(epochs):
    # Convert arrays to tensors
    inputs = torch.tensor(X_train, dtype=torch.float32)
    labels = torch.tensor(y_train, dtype=torch.float32)

    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, labels.unsqueeze(1))

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [10/1000], Loss: 0.6985
Epoch [20/1000], Loss: 0.6639
Epoch [30/1000], Loss: 0.6299
Epoch [40/1000], Loss: 0.5959
Epoch [50/1000], Loss: 0.5625
Epoch [60/1000], Loss: 0.5318
Epoch [70/1000], Loss: 0.5064
Epoch [80/1000], Loss: 0.4877
Epoch [90/1000], Loss: 0.4746
Epoch [100/1000], Loss: 0.4651
Epoch [110/1000], Loss: 0.4578
Epoch [120/1000], Loss: 0.4518
Epoch [130/1000], Loss: 0.4468
Epoch [140/1000], Loss: 0.4424
Epoch [150/1000], Loss: 0.4385
Epoch [160/1000], Loss: 0.4349
Epoch [170/1000], Loss: 0.4315
Epoch [180/1000], Loss: 0.4284
Epoch [190/1000], Loss: 0.4253
Epoch [200/1000], Loss: 0.4225
Epoch [210/1000], Loss: 0.4196
Epoch [220/1000], Loss: 0.4167
Epoch [230/1000], Loss: 0.4139
Epoch [240/1000], Loss: 0.4110
Epoch [250/1000], Loss: 0.4080
Epoch [260/1000], Loss: 0.4049
Epoch [270/1000], Loss: 0.4016
Epoch [280/1000], Loss: 0.3981
Epoch [290/1000], Loss: 0.3946
Epoch [300/1000], Loss: 0.3908
Epoch [310/1000], Loss: 0.3867
Epoch [320/1000], Loss: 0.3822
Epoch [330/1000],

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

with torch.no_grad():
    y_predicted = model(torch.tensor(X_test, dtype=torch.float32))
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(torch.tensor(y_test).unsqueeze(1)).sum() / float(y_test.shape[0])
    precision = precision_score(y_test, y_predicted_cls)
    recall = recall_score(y_test, y_predicted_cls)
    f1 = f1_score(y_test, y_predicted_cls)
    print(f'Accuracy: {acc:.4f}')
    print(f"Test Precision: {precision:.4f}")
    print(f"Test Recall: {recall:.4f}")
    print(f"Test F1-Score: {f1:.4f}")


Accuracy: 0.8580
Test Precision: 0.7023
Test Recall: 0.5185
Test F1-Score: 0.5966
